#### action
文本抄袭自动检测分析:
如果你是某新闻单位工作人员（这里假设source=新华社），为了防止其他媒体抄袭你的文章，你打算做一个抄袭自动检测分析的工具
1）定义可能抄袭的文章来源
2）与原文对比定位抄袭的地方
原始数据：sqlResult.csv，共计89611篇
从数据库导出的文章，字段包括：id, author, source, content, feature, title, url
常用中文停用词：chinese_stopwords.txt

In [3]:
import pandas as pd
import numpy as np
import jieba

In [5]:
#数据加载与预览

artical = pd.read_csv(r'E:\bi_course\L3\tue_course\sqlResult.csv', encoding = 'gb18030')
print(artical.shape)

#文章内容为空的数量
print("文章内容为空的数量:", len(artical[artical.content.isna()]) )

#删除文章内容为空
artical.dropna(subset = ['content'], inplace = True)
print("删除空内容的文章后数量:", artical.shape[0])

(89611, 7)
文章内容为空的数量: 2557
删除空内容的文章后数量: 87054


In [7]:
#对文章进行分词处理

f = open(r"E:\bi_course\L3\tue_course\chinese_stopwords.txt","r", encoding = 'utf-8')
stopwords = [ word[:-1] for  word in f.readlines()] #[:-1]表示删除\n字符

def split_text(text):
    text = text.replace(' ','').replace('\r','').replace('\n','') #\r 回车, \n换行
    words = jieba.cut(text) 
    result  = ' '.join([w for  w in words if w not in stopwords] )
    return result

In [8]:
corpus = list( artical.content.apply( lambda x : split_text(x)) )

Building prefix dict from the default dictionary ...
Loading model from cache D:\UserData\Zoni\Temp\jieba.cache
Loading model cost 1.014 seconds.
Prefix dict has been built succesfully.


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#计算语料库corpus的TD-IDF

cv = CountVectorizer(encoding = 'gb18030', min_df= 0.015)
tfiftransformer = TfidfTransformer()
contervector = cv.fit_transform(corpus)
# print(contervector)
tfdif = tfiftransformer.fit_transform(contervector)


In [58]:
#标记是否为新华社的新闻
label = list(artical.source.apply(lambda x: 1 if '新华社' in str(x)  else 0))

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

X_train, X_test, y_train, y_test = train_test_split(tfdif.toarray(), label, train_size = 0.7)
clf = BernoulliNB()

clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)


print('准确率:', round(accuracy_score(y_test, y_predict),4))
print('精确率:', round(precision_score(y_test, y_predict),4))
print('召回率:', round(recall_score(y_test, y_predict),4))

准确率: 0.8285
精确率: 0.9661
召回率: 0.8397


In [60]:
#使用模型进行风格预测

prediction = clf.predict(tfdif.toarray())
actual = np.array(label)

#比较预测风格和是否为新华社文章:
comparison = pd.DataFrame({'prediction':prediction, 'actual':actual})
comparison['copy'] = 0
comparison.loc[(comparison['prediction'] == 1 ) & (comparison['actual'] == 0), 'copy'] = 1 #预测文章是新华社 实际不是新华社, 则有抄袭嫌疑
copy_article  = comparison[comparison['copy'] == 1]
copy_article.index = copy_article.index

print('可能抄袭文章数:', len(copy_article))

可能抄袭文章数: 2265


In [61]:
copy_article.index

Int64Index([   0,    1,    2,   10,   15,   16,   18,   24,   25,   28,
            ...
            8516, 8523, 8531, 8534, 8535, 8538, 8539, 8546, 8549, 8553],
           dtype='int64', length=2265)

In [13]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfdif.toarray())

#使用聚类对全部文章进行聚类: 分为20类

km = KMeans( n_clusters= 20)
k_labels = km.fit_predict(scaled_array)
print(k_labels.shape)

(87054,)


In [62]:
#创建id_class

id_class = {index: class_ for index, class_ in enumerate (k_labels)} #文章经过kmeans的分类

from collections import defaultdict
class_id = defaultdict(set)

for index, class_ in id_class.items():
    if index in xinhuashe_article.index.to_list():
        class_id[class_].add(index)

In [63]:
from sklearn.metrics.pairwise import cosine_similarity
#查找相似文章

def search_similar_article ( cpindex, top = 10):
    #只在新华社发布的文章中找
    dist_dict = {i: cosine_similarity(tfdif[cpindex], tfdif[i]) for i in class_id [id_class[cpindex]]}
    return sorted (dist_dict.items(), key = lambda x:x[1][0], reverse= True)[:top]

In [68]:
cpindex = 49540
print('是否新华社文章:', cpindex in xinhuashe_article_index)
print('是否抄袭文章:', cpindex in copy_article.index )

print('='*40)

similar_list = search_similar_article(cpindex)
print(similar_list)

print('='*40)

print('怀疑抄袭:\n', artical.iloc[cpindex].content)


print('='*40)
#找一篇相似的原文
similar2 = similar_list[0][0]
print('相似的原文:\n', artical.iloc[similar2].content)

是否新华社文章: True
是否抄袭文章: False
[(49540, array([[1.]])), (49796, array([[0.97036236]])), (48683, array([[0.55479432]])), (49611, array([[0.54022663]])), (52355, array([[0.49310926]])), (47376, array([[0.48170242]])), (48628, array([[0.42808007]])), (48610, array([[0.41713253]])), (52516, array([[0.40458922]])), (48303, array([[0.40101725]]))]
怀疑抄袭:
 　　新华社大马士革５月６日电（记者车宏亮）据叙利亚《祖国报》网站６日报道，在叙利亚建立“冲突降级区”协议于当天零时开始生效。\n　　俄罗斯近日提出在叙西北部伊德利卜省、中部霍姆斯省、大马士革郊区和叙南部地区分别设立“冲突降级区”，在其周边划定安全线、设立检查站，防止冲突发生。俄罗斯、土耳其和伊朗将作为“冲突降级区”担保国，可能出兵监督区内停火执行情况，此外，极端组织势力不得在区内活动。\n　　俄土伊三国代表４日在哈萨克斯坦首都阿斯塔纳签署了关于在叙利亚建立“冲突降级区”的备忘录。对此俄方政要认为，切实建立“冲突降级区”有助于叙政府和反对派保持停火并适时启动政治和谈。\n　　联合国秘书长古特雷斯４日通过发言人发表声明，对俄土伊三国就在叙利亚建立“冲突降级区”达成协议感到鼓舞，欢迎各方重申政治解决叙利亚问题的重要性。（完）\n
相似的原文:
 　　新华社大马士革５月６日电（记者车宏亮）据叙利亚《祖国报》网站６日报道，在叙利亚建立“冲突降级区”协议于当天零时开始生效。\n　　俄罗斯近日提出在叙西北部伊德利卜省、中部霍姆斯省、大马士革郊区和叙南部地区分别设立“冲突降级区”，在其周边划定安全线、设立检查站，防止冲突发生。俄罗斯、土耳其和伊朗将作为“冲突降级区”担保国，可能出兵监督区内停火执行情况，此外，极端组织势力不得在区内活动。\n　　俄土伊三国代表４日在哈萨克斯坦首都阿斯塔纳签署了关于在叙利亚建立“冲突降级区”的备忘录。对此俄方政要认为，切实建立“冲突降级区”有助于叙政府和反对派保持停火并